# Set new champion model

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

EXPERIMENT_NAME = "apartment-price-prediction"
CANDIDATE_MODEL_NAME = "candidate-model"
CHAMPION_MODEL_NAME = "champion-model"
BEST_METRIC = "mse"  # Change if needed

mlflow.set_tracking_uri("http://135.181.85.42:5000")

client = MlflowClient()

# 1. Get experiment
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found.")

# 2. Get best run based on lowest MSE
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="attributes.status = 'FINISHED'",
    order_by=[f"metrics.{BEST_METRIC} ASC"],
    max_results=1
)

if not runs:
    raise ValueError("No finished runs found with the required metric.")

best_run = runs[0]
best_run_id = best_run.info.run_id
best_metric_value = best_run.data.metrics[BEST_METRIC]

# 3. Find the registered model version corresponding to that run (in "candidate-model")
candidate_versions = client.search_model_versions(
    f"name='{CANDIDATE_MODEL_NAME}'")
best_version = None

for version in candidate_versions:
    if version.run_id == best_run_id:
        best_version = version
        break

if not best_version:
    raise ValueError(
        f"No registered version of '{CANDIDATE_MODEL_NAME}' found for run {best_run_id}")

print(
    f"Promoting version {best_version.version} (mse={best_metric_value}) to Production as '{CHAMPION_MODEL_NAME}'.")

# 4. Register or update model under "championModel"
# First: check if this version is already registered under "championModel"
existing_champion_versions = client.search_model_versions(
    f"name='{CHAMPION_MODEL_NAME}'")
existing_version_with_run = next(
    (v for v in existing_champion_versions if v.run_id == best_run_id), None)

if not existing_version_with_run:
    # Register new version
    result = mlflow.register_model(
        model_uri=f"runs:/{best_run_id}/model",
        name=CHAMPION_MODEL_NAME,
    )
    champion_version = result.version
else:
    champion_version = existing_version_with_run.version

# 5. Promote to production
client.transition_model_version_stage(
    name=CHAMPION_MODEL_NAME,
    version=champion_version,
    stage="Production",
    archive_existing_versions=True,
)

print(
    f"✔️ Model version {champion_version} promoted to 'Production' in '{CHAMPION_MODEL_NAME}'.")

Promoting version 1 (mse=623940.4181705144) to Production as 'champion-model'.


/var/folders/74/sn92zv5x10396pnw3vtmq68m0000gn/T/ipykernel_20668/4256598909.py:68: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


✔️ Model version 1 promoted to 'Production' in 'champion-model'.
